In [4]:
!pip install flask jupyter_client


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install flask-cors


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import openai
print(openai.__version__)

1.28.1


In [8]:
pip install python-pptx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install openpyxl


  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/6a/94/a59521de836ef0da54aaf50da6c4da8fb4072fb3053fa71f052fd9399e7a/openpyxl-3.1.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/96/c2/3dd434b0108730014f1b96fd286040dc3bcb70066346f7e01ec2ac95865f/et_xmlfile-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   -------------- ------------------------- 92.2/250.0 kB 2.6 MB/s eta 0:00:01
   ------------------------------------ --- 225.3/250.0 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
import jupyter_client
from flask import Flask, request, jsonify, render_template, send_file
from flask_cors import CORS
from pptx import Presentation
from pptx.util import Pt
import os
import io
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from openai import OpenAI
from openpyxl import Workbook

In [56]:
app = Flask(__name__)
# CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})
CORS(app)

In [57]:
@app.route('/send-gen-ppt', methods=['POST'])
def receive_message():
    data = request.json  # Assuming JSON payload
    if not data:
        return {"error": "No data received."}
    ans = data['searchQuery']
    print(ans)
    api_key = "sk-proj-TYcssHY0HC7h0uTnCDI4T3BlbkFJtZa3DvaZLAJctiACaze2"  # Fetch your API key from environment variables
    if not api_key:
        return {"error": "OpenAI API key not found."}

    print(api_key)
    client = OpenAI(api_key=api_key)

    TEMPLATE_FILE = "D://hackblr//frontend--search//pages//notebooks//template.pptx"
    if not os.path.exists(TEMPLATE_FILE):
        return {"error": "Template file not found."}

    presentation_title="ABCD"
    slide_contents = ans

    user_message_content = f"Create a presentation with 3 slides.\n" \
                           f"Presentation title: {presentation_title}\n" \
                           f"Slide content: {slide_contents}\n"

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an assistant that gives ideas for PowerPoint presentations."},
            {"role": "user", "content": user_message_content},
        ],
        model="gpt-3.5-turbo",
    )
    print(chat_completion.choices[0].message.content)
    if chat_completion.choices[0].message.content:
        generated_text = chat_completion.choices[0].message.content

        presentation = Presentation(TEMPLATE_FILE)

        slides = generated_text.split("Slide")

        if len(slides) - 1 <= len(presentation.slides):
            for i, slide_content in enumerate(slides[1:], start=0):
                content_lines = slide_content.strip().split("\n")
                if len(content_lines) >= 2:
                    slide_title = content_lines[0].strip()
                    slide_content = content_lines[1:]  # Extract bullet points

                    slide = presentation.slides[i]
                    title = slide.shapes.title
                    content_box = slide.placeholders[1]

                    title.text = slide_title
                    title.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)

                    content_box.text = ""  # Clear existing text
                    content_frame = content_box.text_frame
                    content_frame.clear()  # Clear existing paragraphs
                    for point in slide_content:
                        p = content_frame.add_paragraph()
                        p.text = point.strip()
                        p.font.color.rgb = RGBColor(255, 255, 255)
                        p.space_after = Pt(14)  # Adjust spacing between bullet points

        else:
            return {"error": "The template has fewer slides than required."}


        # Save the PowerPoint presentation in the local directory
        

#         pptx_bytes = io.BytesIO()
#         presentation.save(pptx_bytes)
#         pptx_bytes.seek(0)

#         with open('generated_presentation.pptx', 'wb') as f:
#             f.write(pptx_bytes.read())





#         pptx_bytes = io.BytesIO()
#         presentation.save(pptx_bytes)
#         pptx_bytes.seek(0)

#         return send_file(
#                 pptx_bytes,
#                 mimetype='application/vnd.openxmlformats-officedocument.presentationml.presentation',
#                 as_attachment=True,
#                 download_name=f"{presentation_title}.pptx"
# )

    else:
        return {"error": "No response from OpenAI."}



    
    return jsonify({"message": "Search query received successfully"})
    # if 'searchQuery' in data:
    #     search_query = data['searchQuery']
    #     print(search_query)
    #     # Process the search query here
    #     return jsonify({"message": "Search query received successfully"})
    # else:
    #     return jsonify({"error": "Missing 'searchQuery' field in JSON payload"}), 400

@app.route("/revert-gen-ppt", methods=['GET'])
def return_home():
    return jsonify({
        "message" : "Message-Gone"
    })
app.run(debug=True, use_reloader=False, port=8000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [11/May/2024 23:02:19] "GET / HTTP/1.1" 200 -


Something about humans and their will to survive
sk-proj-TYcssHY0HC7h0uTnCDI4T3BlbkFJtZa3DvaZLAJctiACaze2


127.0.0.1 - - [11/May/2024 23:02:30] "POST /send-gen-ppt HTTP/1.1" 200 -


Slide 1:
Title: The Power of Survival
Content:
- Humans are remarkable creatures who possess a strong will to survive in challenging circumstances.
- Our ability to adapt, innovate, and persevere in the face of adversity is a testament to our resilience as a species.

Slide 2:
Title: Survival Instincts
Content:
- From the earliest days of our evolution, humans have developed survival instincts that drive us to seek food, shelter, and safety.
- Our fight-or-flight response, critical thinking skills, and social cooperation have all played a role in our ability to overcome obstacles and thrive.

Slide 3:
Title: Lessons from the Past
Content:
- Throughout history, humans have faced and survived countless hardships, from natural disasters to wars and pandemics.
- By learning from our past experiences and drawing inspiration from our ancestors, we can continue to navigate challenges and build a better future for generations to come.
